In [212]:
import os
import sys
import numpy as np
from random import randint

class HMM(object): 
    # base class for different HMM models
    def __init__(self, M, N):
        # model is (A, B, pi) where A = Transition probs(hidden_states*hidden_states), 
        #B = Emission Probs(hidden_states*states), pi = initial distribution(hidden_states)        
        # M:number of hidden states of the model
        # N:number of states of the model
        self.A_raw = np.random.random((M, M)) 
        self.row_sums_A = self.A_raw.sum(axis=1)
        self.A = self.A_raw / self.row_sums_A[:, np.newaxis]
        # Get transition probability
        self.B_raw = np.random.random((M, N))
        self.row_sums_B = self.B_raw.sum(axis=1)
        self.B = self.B_raw / self.row_sums_B[:, np.newaxis]
        # Get emission probability
        self.pi_raw = np.random.random((1, M)) 
        self.row_sums_pi = self.pi_raw.sum(axis=1)
        self.pi = self.pi_raw / self.row_sums_pi[:, np.newaxis]
        # Get initial probability
        self.M = M
        self.N = N

        
    def backward(self, obs):
        # This function is for backward algorithm, suppose that A, B, pi 
        #are given, and it calculates a bwk matrix (obs*states).
        # The backward algorithm can be used to calculate the likelihood 
        #of the probability P(Y_{k+1}, ... , Y_n|t_k=C)
        #=sum_q P(Y_{k+2}, ... , Y_n|t_{k+1}=q)P(q|C)P(x_{k+1}|q)
        #The backward probability b is the probability of seeing the observations from 
        #time t + 1 to the end, given that we are in state i at time t
        self.bwk = [[0 for x in range(self.M)] for y in range(len(obs))] 
        # Initalize bwk to be empty matrix T*M
        # Initialize base cases (t == T)
        for y in range(self.M):
            self.bwk[len(obs)-1][y] = 1 
        for t in reversed(range(len(obs)-1)):
            for y in range(self.M):
                self.bwk[t][y] = sum((self.bwk[t+1][y1] * self.A[y][y1] * self.B[y1][obs[t+1]]) 
                                    for y1 in range(self.M))
                #beta_k(C)=\sum_q beta_{k+1}(q)P(q|C)P(w_{k+1}|q)
        prob = sum((self.pi[0][y]* self.B[y][obs[0]] * self.bwk[0][y]) for y in range(self.M))

        return prob 
        #This prob is the likelihood of the input obs   
 

    def forward(self, obs):
        # This function is for forward algorithm, suppose that A, B, pi are given, 
        #and it calculates a fwd matrix (obs*states).
        # The forward algorithm can be used to calculate the likelihood of the model
        #P(Y1, ... , Yn)=sum_t(\prod_i P(Y[i]|t[i])P(t[i]|t[i-1])
        self.fwd = [[0 for x in range(self.M)] for y in range(len(obs))]  
        #Initalize fwk to be empty matrix, and finally fwd is T*M
        # Initialize base cases (t == 0)
        for y in range(self.M):
            self.fwd[0][y] = self.pi[0][y] * self.B[y][obs[0]] 
            #alpha_1(q)=p(w1,t1=q)=P(t1=q|t0)*p(w1|t1=q)
        # Run Forward algorithm for t > 0
        for t in range(1, len(obs)):
            for y in range(self.M):
                self.fwd[t][y] = sum((self.fwd[t-1][y0] * self.A[y0][y] * self.B[y][obs[t]]) 
                                     for y0 in range(self.M))
                #alpha_k(q)=\sum_q1 alpha_{k-1}(q1)P(t_k=q|t_{k-1}=q1)P(w_k|t+k=q)
        prob = sum((self.fwd[len(obs) - 1][s]) for s in range(self.M))
        # The likelihood of input equals to the summation of fwd[N][t]
        return prob

    
    def viterbi(self, obs):
    #the task of determining which sequence of variables is the underlying source 
    #of some sequence of observations is called the decoding task
    #Decoding: Given as input an HMM = (A, B, pi) and a sequence of observations 
    #O = Y_1, ... Y_N, find the most probable sequence of states Q = X_1, ... X_T.
    # Goal: find the best path!
    # argmax_t P(Y1, ... Y_N, X_1, ..., X_T|A, B, pi)
        vit = [[0 for x in range(self.M)] for y in range(len(obs))] 
        # matrix
        path = {} 
        # path
        # Initialize base cases (t == 0)
        for y in range(self.M):
            vit[0][y] = self.pi[0][y] * self.B[y][obs[0]]
            path[y] = [y]
        
        # Run Viterbi for t > 0
        for t in range(1, len(obs)):
            newpath = {}
            for y in range(self.M):
                (prob, state) = max((vit[t-1][y0] * self.A[y0][y] * self.B[y][obs[t]], y0) 
                                    for y0 in range(self.M))
                vit[t][y] = prob
                newpath[y] = path[state] + [y]
            # Don't need to remember the old paths
            path = newpath
        n = 0           
        # if only one element is observed max is sought in the initialization values
        if len(obs)!=1:
            n = t
        (prob, state) = max((vit[n][y], y) for y in range(self.M))
        return (prob, path[state])

    
    
    def forward_backward(self, obs): 
        #Output matrix gamma: gamma[t][y]=P(q_t=j|Y_1, ..., Y_N,A,B,pi)
        #and tensor zi: zi[t][i][j]=P(q_t=i,q_{t+1}=j|Y_1, ..., Y_N,A,B,pi)
        gamma = [[0 for x in range(self.M)] for y in range(len(obs))]
        # this is needed to keep track of finding a state i at a time t for all i and all t
        zi = [[[0 for x in range(self.M)] for y in range(self.M)] for z in range(len(obs))]  
        # this is needed to keep track of finding a state i at a time t and j at a time (t+1) 
        #for all i and all j and all t
        # get alpha and beta tables computes
        p_obs = self.forward(obs)
        self.backward(obs)
        # compute gamma values
        for t in range(len(obs)):
            for y in range(self.M):
                gamma[t][y] = (self.fwd[t][y] * self.bwk[t][y]) / p_obs
                if t == 0:
                    self.pi[0][y] = gamma[t][y]
                #gamma[t][y]=P(q_t=j|Y_1, ..., Y_N,A,B,pi)
                #=P(q_t=j,Y_1, ..., Y_N|A,B,pi)/P(Y_1, ..., Y_N|A,B,pi)
                #=alpha_t(j)beta_t(j)/P(Y_1, ..., Y_N|A,B,pi)
                #compute zi values up to T - 1
                if t == len(obs) - 1:
                    continue
                for y1 in range(self.M):
                    zi[t][y][y1] = self.fwd[t][y] * self.A[y][y1] * self.B[y1][obs[t + 1]] * self.bwk[t + 1][y1] / p_obs
        #zi[t][i][j]=P(q_t=i,q_{t+1}=j|Y_1, ..., Y_N,A,B,pi)
        #=P(q_t=i,q_{t+1}=j,Y_1, ..., Y_N|A,B,pi)/P(Y_1, ..., Y_N|A,B,pi)
        #=alpha_t(i)a_{ij}b_j(O_{t+1})beta_{t+1}(j)/apha_t(X_T)
        return (gamma,zi)
    
    
    def baum_welch(self,obs):
        # returns model given the initial model and observations  
        #The Baum-Welch algorithm iteratively estimate the counts.
        #We will start with an estimate for the transition and observation probabilities and 
        #then use these estimated probabilities to derive better and better probabilities. 
        #We get our estimated probabilities by computing the forward probability for 
        #an observation and then dividing that probability mass among all the different 
        #paths that contributed to this forward probability.
        gamma = [[0 for x in range(self.M)] for y in range(len(obs))]
        zi = [[[0 for x in range(self.M)] for y in range(self.M)] for z in range(len(obs))]  

        # now that we have gamma and zi let us re-estimate
        (gamma,zi)=self.forward_backward(obs)
        for y in range(self.M):
            for y1 in range(self.M):
                # we will now compute new a_ij
                #a_{ij)=expected number of transitions from state i to state j/expected number 
                #of transitions from state i
                val = sum([zi[t][y][y1] for t in range(len(obs) - 1)]) #
                val /= sum([gamma[t][y] for t in range(len(obs) - 1)])
                self.A[y][y1] = val
        # re estimate gamma
        for y in range(self.M):
            for k in range(self.N): 
                # for all symbols vk
                val = 0.0
                for t in range(len(obs)):
                    if obs[t] == k :
                        val += gamma[t][y]
                val /= sum([gamma[t][y] for t in range(len(obs))])
                self.B[y][k] = val
                    #b_j(v_k)=expected number of times in state j and observing symbol vk/expected 
                    #number of times in state j
        return

In [213]:
M=randint(0,10)
N=randint(0,10)
hmm=HMM(M,N)
T=randint(0,10)
observations = []
for i in xrange(0,T):
    observations.append(randint(0,N-1))
#observations=[1,0,1,1]
print "M=", M, "N=", N, "Observations = ", observations

M= 5 N= 2 Observations =  [0, 1, 1]


In [214]:
p1=hmm.backward(observations)
print " Fwd Prob = ", p1

 Fwd Prob =  0.103988308511


In [215]:
p2=hmm.forward(observations)
print " Fwd Prob = ", p2

 Fwd Prob =  0.103988308511


In [216]:
prob, hidden_states = hmm.viterbi(observations)
print "Max Probability = ", prob, " Hidden State Sequence = ", hidden_states

Max Probability =  0.00537607558932  Hidden State Sequence =  [4, 0, 2]


In [217]:
(gamma,zi)=hmm.forward_backward(observations)
print "zi= ", zi
print "gamma=", gamma

zi=  [[[0.0011624564957250247, 0.0020030545878707193, 0.015631611790854025, 0.0091444324785135821, 0.0043859720749167518], [0.015426267521049362, 0.032237269649808588, 0.0070780183064178678, 0.032613453218488672, 0.010794374784312272], [0.044725449672599456, 0.019152057548830908, 0.045605457751467848, 0.01366066705866585, 0.019639764148948406], [0.052406826763653894, 0.015469885862274579, 0.091177137670446703, 0.086442060891582531, 0.13085897811145911], [0.10621852622921586, 0.12981980523480724, 0.00059821685199990568, 0.061711583659758838, 0.052036671636331966]], [[0.0076353920449904397, 0.012629478820593809, 0.10704930873531725, 0.063637799323571401, 0.028987547757770706], [0.030906681961741615, 0.061999443015460322, 0.014785250261079108, 0.069229639978866081, 0.021761057666444907], [0.049416064375145827, 0.020312664148362337, 0.052535846147762987, 0.015991496521152962, 0.021834371178762229], [0.027669036381096551, 0.0078402749170779235, 0.050190046557806053, 0.048354293889129651, 0.

In [218]:
hmm.baum_welch(observations)
print "The new model parameters after 1 iteration are: "
print "A = ", hmm.A
print "B = ", hmm.B
print "pi = ", hmm.pi

The new model parameters after 1 iteration are: 
A =  [[ 0.03487266  0.0579951   0.48632003  0.28852553  0.13228668]
 [ 0.15606466  0.31719979  0.0736942   0.34337246  0.10966889]
 [ 0.31102085  0.13060807  0.32366246  0.09772406  0.13698457]
 [ 0.13786515  0.04002849  0.24404239  0.23294095  0.34512302]
 [ 0.30346999  0.36622441  0.00173204  0.17961233  0.14896123]]
B =  [[ 0.07648448  0.92351552]
 [ 0.20333554  0.79666446]
 [ 0.28740867  0.71259133]
 [ 0.38659414  0.61340586]
 [ 0.53887609  0.46112391]]
pi =  [[ 0.03383755  0.10446205  0.14767661  0.28044915  0.43357465]]
